In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
    # Set the default device to GPU
    torch.cuda.set_device(0)
else:
    print("CUDA is not available. Running on CPU.")


CUDA is available. GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
# !pip install -q git+https://github.com/huggingface/transformers.git
# !pip install transformers datasets torch
#!pip install imgaug
#!pip install seqeval
#!pip install transformers[torch]
# !pip install accelerate -U

In [3]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification,AutoProcessor
from PIL import Image, ImageDraw
import numpy as np
import tqdm as notebook_tqdm
import json
import pandas as pd
import os
from pdf2image import convert_from_path
import fitz  # PyMuPDF
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import io
from PIL import Image, ImageDraw, ImageFont
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from torch.utils.data import DataLoader, Dataset
from datasets import load_metric
from torch.optim import AdamW
from torch.utils.data import DataLoader

C:\Users\makni\anaconda3\envs\PFA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def read_annotation_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [5]:
# annotation_data= read_annotation_file("C:/Users/makni/Downloads/ieee/preprocessed_annotations/WRO-1.pdf.json")

# # annotation_data

In [6]:
# from PIL import Image, ImageDraw
# import json

# def display_image_with_annotations(image_path, json_path, bbox_color='red'):
#     # Load the image and convert to RGB
#     image = Image.open(image_path).convert("RGB")
#     print(image.size)
#     draw = ImageDraw.Draw(image)
    
#     # Load the annotation data
#     with open(json_path, 'r') as file:
#         annotations = json.load(file)
    
#     # Iterate through the annotations and draw rectangles
#     for word, bbox in zip(annotations['words'], annotations['bboxes']):
#         draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], outline=bbox_color, width=2)
#         draw.text((bbox[0] + 2, bbox[1] + 2), word, fill='blue')
    
#     # Display the image
#     image.show()


# # Test the function
# annotation_path = "C:/Users/makni/Downloads/ieee/preprocessed_annotations/20231217_190133_merged-10.pdf.json"
# image_path = "C:/Users/makni/Downloads/ieee/preprocessed_images/20231217_190133_merged-10.pdf_1.png"
# display_image_with_annotations(image_path, annotation_path)  # Change bbox_color to any color you like


# Data Augmentation

In [7]:
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
def get_train_transform():
    return A.Compose([
        A.MedianBlur(blur_limit=3, p=0.3),
        A.ColorJitter(p=0.3),
        A.PadIfNeeded(min_height=1000, min_width=700, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255], p=1.0),
        A.Rotate(limit=5, p=1, interpolation=cv2.INTER_CUBIC),
    ], bbox_params=A.BboxParams(
        format='pascal_voc',
        label_fields=['labels'],
        min_area=0,
        min_visibility=0.1  
    ))

In [8]:
# !pip install albumentations

# LayoutLmv3

## Label Dictionaries:

In [9]:
label2id = {
    'O': 0,
    'B-title': 1,
    'I-title': 2,
    'B-date': 3,
    'I-date': 4,
    'B-ieee': 5,
    'I-ieee': 6,
    'B-total': 7,
    'I-total': 8,
    'B-totalValue': 9,
    'I-totalValue': 10
}

# Create id2label by reversing label2id
id2label = {id: label for label, id in label2id.items()}

## Processor Initializationng

In [10]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

## Custom Dataset Class:

In [11]:
# def crop_to_bounding_boxes(image_np, boxes):
#         if not boxes:
#             return image_np  # Return original if no boxes
    
#         # Initialize min and max coordinates with values from the first box
#         x_min = boxes[0][0]
#         y_min = boxes[0][1]
#         x_max = boxes[0][2]
#         y_max = boxes[0][3]
    
#         # Iterate over all boxes to find the minimum and maximum extents
#         for box in boxes:
#             x_min = min(x_min, box[0])
#             y_min = min(y_min, box[1])
#             x_max = max(x_max, box[2])
#             y_max = max(y_max, box[3])
    
#         # Crop the image using the min and max coordinates
#         image_cropped = image_np[y_min-20:y_max+20, x_min-20:x_max+20]
    
#         adjusted_boxes = []
#         for box in boxes:
#             adjusted_box = [
#                 box[0] - x_min+20 ,
#                 box[1] - y_min+20,
#                 box[2] - x_min+20,
#                 box[3] - y_min+20
#             ]
#             adjusted_boxes.append(adjusted_box)
    
#         return image_cropped, adjusted_boxes


In [12]:
# import numpy as np
# from PIL import Image
# import torch
# from torch.utils.data import Dataset

# class CustomDataset(Dataset):
#     def __init__(self, annotation_dir, image_dir, annotation_files, transforms=None):
#         self.annotations = [os.path.join(annotation_dir, file) for file in annotation_files]
#         self.image_dir = image_dir
#         self.processor = processor  
#         self.transforms = transforms

#     def __getitem__(self, idx):
#         annotation_path = self.annotations[idx]
#         with open(annotation_path, 'r') as f:
#             data = json.load(f)

#         image_path = os.path.join(self.image_dir, os.path.basename(data["path"]))
#         image = Image.open(image_path).convert("RGB")

#         width, height = image.size

#         words = data["words"]
#         labels = [label2id[label] for label in data["ner_tags"]]
#         boxes = data["bboxes"]

#         image_np = np.array(image)  # Convert PIL Image to NumPy array for transformation
#         # print(f"Before transformation: {len(words)} words, {len(boxes)} boxes")

#         if self.transforms:
#             transformed = self.transforms(image=image_np, bboxes=boxes, labels=labels)
#             image_np = transformed['image']
#             boxes = transformed['bboxes']
#             labels = transformed['labels']
            
#         boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in boxes]
#         cropped_image, boxes = crop_to_bounding_boxes(image_np, boxes)

#         image = Image.fromarray(cropped_image)  
#         image = image.resize((600, 900))
#         width, height = image.size
        
#         boxes = [[bbox[0] / width, bbox[1] / height, bbox[2] / width, bbox[3] / height] for bbox in boxes]
#         boxes = [[int(box[0] * 1000), int(box[1] * 1000), int(box[2] * 1000), int(box[3] * 1000)] for box in boxes]

#         image.convert("RGB")
#         encoding = self.processor(image, words, boxes=boxes, word_labels=labels, truncation=True, padding="max_length", return_tensors="pt")
#         bbox_tensor = torch.tensor(boxes, dtype=torch.long)
#         label_tensor = torch.tensor(labels, dtype=torch.long)
        
#         return {
#             'words': words,
#             'input_ids': encoding['input_ids'].squeeze(0),
#             'attention_mask': encoding['attention_mask'].squeeze(0),
#             'bbox': bbox_tensor,  
#             'labels': label_tensor
#         }

#     def __len__(self):
#         return len(self.annotations)

#     def check_and_adjust_boxes(self, boxes, img_width, img_height):
#         adjusted_boxes = []
#         for box in boxes:
#             x_min, y_min, x_max, y_max = box
#             if x_min < 0 or y_min < 0 or x_max > img_width or y_max > img_height:
#                 # Box is out of bounds, adjust or discard
#                 x_min = max(0, x_min)
#                 y_min = max(0, y_min)
#                 x_max = min(img_width, x_max)
#                 y_max = min(img_height, y_max)
#             adjusted_boxes.append([x_min, y_min, x_max, y_max])
#         return adjusted_boxes

    

In [13]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch import clamp

def crop_to_bounding_boxes(image_np, boxes):
        if not boxes:
            return image_np  # Return original if no boxes
    
        # Initialize min and max coordinates with values from the first box
        x_min = boxes[0][0]
        y_min = boxes[0][1]
        x_max = boxes[0][2]
        y_max = boxes[0][3]
    
        # Iterate over all boxes to find the minimum and maximum extents
        for box in boxes:
            x_min = min(x_min, box[0])
            y_min = min(y_min, box[1])
            x_max = max(x_max, box[2])
            y_max = max(y_max, box[3])
    
        # Crop the image using the min and max coordinates
        image_cropped = image_np[y_min-20:y_max+20, x_min-20:x_max+20]
    
        adjusted_boxes = []
        for box in boxes:
            adjusted_box = [
                box[0] - x_min+20 ,
                box[1] - y_min+20,
                box[2] - x_min+20,
                box[3] - y_min+20
            ]
            adjusted_boxes.append(adjusted_box)
    
        return image_cropped, adjusted_boxes
def clamp_values(value, min_value, max_value):
    return max(min_value, min(value, max_value))

class CustomDataset(Dataset):
    def __init__(self, annotation_dir, image_dir, annotation_files, transforms=None):
        self.annotations = [os.path.join(annotation_dir, file) for file in annotation_files]
        self.image_dir = image_dir
        self.processor = processor  
        self.transforms = transforms

    def __getitem__(self, idx):
        annotation_path = self.annotations[idx]
        with open(annotation_path, 'r') as f:
            data = json.load(f)

        image_path = os.path.join(self.image_dir, os.path.basename(data["path"]))
        image = Image.open(image_path).convert("RGB")

        width, height = image.size

        words = data["words"]
        labels = [label2id[label] for label in data["ner_tags"]]
        boxes = data["bboxes"]

        image_np = np.array(image)  # Convert PIL Image to NumPy array for transformation
        # print(f"Before transformation: {len(words)} words, {len(boxes)} boxes")

        if self.transforms:
            transformed = self.transforms(image=image_np, bboxes=boxes, labels=labels)
            image_np = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']


        boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in boxes]
        cropped_image, boxes = crop_to_bounding_boxes(image_np, boxes)

        boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in boxes]


        image = Image.fromarray(cropped_image)  
        image = image.resize((600, 900), resample=Image.BILINEAR)
        # boxes = normalize_bboxes(boxes, 600, 900)
        # boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in boxes]

        boxes = [[bbox[0] / width, bbox[1] / height, bbox[2] / width, bbox[3] / height] for bbox in boxes]
        boxes = [[int(box[0] * 1000), int(box[1] * 1000), int(box[2] * 1000), int(box[3] * 1000)] for box in boxes]
        
        boxes = [[clamp_values(coord, 0, 1000) for coord in box] for box in boxes]

        image.convert("RGB")
        encoding = self.processor(image, words, boxes=boxes, word_labels=labels, truncation=True, padding="max_length", return_tensors="pt")
        bbox_tensor = torch.tensor(boxes, dtype=torch.long)
        label_tensor = torch.tensor(labels, dtype=torch.long)
        
        return {
            'words': words,
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'bbox': bbox_tensor,  
            'labels': label_tensor
        }

    def __len__(self):
        return len(self.annotations)


In [14]:
import numpy as np
from torch.utils.data import DataLoader
annotation_dir = "output/"
image_dir = "resized_images/" 
annotation_files = [file for file in os.listdir(annotation_dir) if file.endswith('.json')]
np.random.shuffle(annotation_files)  
split_idx = int(0.8 * len(annotation_files))
train_files = annotation_files[:split_idx]
test_files = annotation_files[split_idx:]
train_dataset = CustomDataset(annotation_dir, image_dir, annotation_files=train_files,transforms=get_train_transform()) #,transforms=get_train_transform()
test_dataset = CustomDataset(annotation_dir, image_dir, annotation_files=test_files)



In [15]:
# for i in range (1,156):
#     # print(i)
example = train_dataset[1]
# print(example)
# image= example["image"]
# print(image)
words = example["words"]
boxes = example["bbox"]
ner_tags = example["labels"]
    # ids= example["input_ids"]
    # masks= example["attention_mask"]
    # print("masks:", masks)
    # print("ids:", ids)
    
    # print(len(words),boxes.size() ,ner_tags.size() )
    # print("Bounding Boxes:", boxes.size())
    # # print("Bounding Boxes:", boxes)
    # print("NER Tags:", ner_tags.size())
    # print(boxes.shape)
    # print(ner_tags.shape)
    # print(ner_tags)
    # print(len(words))

In [16]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    max_len = max([len(item['input_ids']) for item in batch])
    input_ids = torch.stack([torch.cat([item['input_ids'], torch.tensor([processor.tokenizer.pad_token_id] * (max_len - len(item['input_ids'])), dtype=torch.long)]) for item in batch])
    attention_mask = torch.stack([torch.cat([item['attention_mask'], torch.tensor([0] * (max_len - len(item['attention_mask'])), dtype=torch.long)]) for item in batch])

    bbox_padded = []
    for item in batch:
        padding_length = max_len - item['bbox'].shape[0]
        padded_bbox = torch.cat([item['bbox'], torch.zeros(padding_length, 4, dtype=torch.int64)])
        bbox_padded.append(padded_bbox)
    bbox = torch.stack(bbox_padded)

    labels = torch.stack([torch.cat([item['labels'], torch.tensor([-100] * (max_len - len(item['labels'])), dtype=torch.long)]) for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'bbox': bbox,
        'labels': labels
    }

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

In [17]:
for batch in train_loader:
    print({k: v.shape for k, v in batch.items()})
    break 

print(train_loader)

{'input_ids': torch.Size([4, 512]), 'attention_mask': torch.Size([4, 512]), 'bbox': torch.Size([4, 512, 4]), 'labels': torch.Size([4, 512])}


# Define metrics

# Define the model

In [18]:
model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    id2label=id2label,
    label2id=label2id)



Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import numpy as np
import evaluate

# deprecated 
# from datasets import load_metric
# metric = load_metric("seqeval")

metric = evaluate.load("seqeval")
label_list = list(model.config.id2label.values())


def compute_metrics(p, return_entity_level_metrics=False):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Set zero_division=1 to handle classes with no predictions gracefully
    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=1)
    if return_entity_level_metrics:
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [20]:
# from transformers import LayoutLMv3FeatureExtractor, AutoTokenizer, LayoutLMv3Processor
# from transformers import LiltForTokenClassification
# # huggingface hub model id
# model_id = "SCUT-DLVCLab/lilt-roberta-en-base"
# # load model with correct number of labels and mapping
# model = LiltForTokenClassification.from_pretrained(
#     model_id, num_labels=11, label2id=label2id, id2label=id2label
# )

In [21]:
# # Utilisez cette ligne pour créer l'optimiseur
# optimizer = AdamW(model.parameters(), lr=5e-3, no_deprecation_warning=True)
# # Move the model to the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Define TrainingArguments + Trainer


In [22]:
from transformers import TrainingArguments, Trainer

# MLFlow

In [23]:
!git clone https://github.com/akramLh005/PFA-mlflow.git
!cd PFA-mlflow


fatal: destination path 'PFA-mlflow' already exists and is not an empty directory.


In [24]:
#!pip install mlflow==2.11.1

In [25]:
%env MLFLOW_TRACKING_URI=https://dagshub.com/akramLh005/PFA-mlflow.mlflow
%env MLFLOW_TRACKING_USERNAME=akramLh005
%env MLFLOW_TRACKING_PASSWORD=fabaf87fce39ef7e9bfdd406d6be4354ce2a25f4


env: MLFLOW_TRACKING_URI=https://dagshub.com/akramLh005/PFA-mlflow.mlflow
env: MLFLOW_TRACKING_USERNAME=akramLh005
env: MLFLOW_TRACKING_PASSWORD=fabaf87fce39ef7e9bfdd406d6be4354ce2a25f4


# Train

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [27]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch.nn.utils import clip_grad_norm_

def check_gradient(model):
    def hook(grad):
        if torch.isnan(grad).any() or torch.isinf(grad).any():
            print("NaN or Inf in gradients.")

    for p in model.parameters():
        p.register_hook(hook)



In [28]:
def calculate_class_counts(dataset):
    # Initialiser un dictionnaire pour compter les occurrences de chaque classe
    class_counts = {label: 0 for label in label2id.values()}

    # Parcourir le jeu de données
    for i in range(len(dataset)):
        sample = dataset[i]
        labels = sample['labels'].numpy()  # Assurez-vous que les labels sont sous forme de Numpy array pour comptage
        
        # Compter chaque label
        for label in labels:
            if label != -100:  # Assurez-vous de ne pas compter les labels de padding
                class_counts[label] += 1

    return class_counts

# Appliquez cette fonction à votre jeu de données d'entraînement
class_counts = calculate_class_counts(train_dataset)
class_counts_list = [class_counts[i] for i in range(len(label2id))]  # Transformer en liste pour l'usage ultérieur

print("Comptages de classes :", class_counts_list)


Comptages de classes : [25452, 134, 35, 147, 53, 128, 290, 149, 97, 145, 28]


In [29]:
class_counts = [25989, 137, 40, 146, 39, 133, 316, 150, 99, 147, 26]  # Comptages fournis
total_count = sum(class_counts)
class_weights = [total_count / count for count in class_counts]

# Convertir en tensor de poids pour PyTorch
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

total_count = sum(class_counts)
class_weights = [total_count / (count + 1e-5) for count in class_counts]  # Small epsilon to avoid division by zero


# Afficher les poids calculés pour chaque classe
print("Poids pour chaque classe :", class_weights)
from torch import nn
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
model.to(device)


Poids pour chaque classe : [1.047443148621554, 198.70071542330544, 680.5498298625424, 186.45204202383275, 697.9998210256869, 204.6766763400995, 86.14556689412764, 181.47998790133414, 274.9696691949829, 185.18366087185981, 1046.9995973078471]


LayoutLMv3ForTokenClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Encoder

In [30]:
loss_func = nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)

In [31]:
import torch
torch.autograd.set_detect_anomaly(True)


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


LayoutLMv3ForTokenClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Encoder

In [33]:
from transformers import TrainingArguments, Trainer, AdamW, get_scheduler
import mlflow
from transformers import EarlyStoppingCallback

from torch.optim import AdamW 

weight_decay_rate = 0.01
mlflow.end_run()
training_args = TrainingArguments(
    output_dir="test_trainer",
    max_grad_norm=1.0, 
    
    evaluation_strategy="steps",
    eval_steps=50,
    
    save_strategy="steps",
    save_steps=200,

    
    save_total_limit=2,
    num_train_epochs=50,
    learning_rate=5e-5,

    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=200,

    weight_decay = 0.01,

    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    max_steps=2000,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

class MLflowTrainer(Trainer):
    def __init__(self, class_weights_tensor, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_func = nn.CrossEntropyLoss(weight=class_weights_tensor, ignore_index=-100)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = self.loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

    def training_step(self, model, inputs):
        model.train()
        inputs = self._prepare_inputs(inputs)
        with self.autocast_smart_context_manager():
            loss = self.compute_loss(model, inputs)
        if self.args.gradient_accumulation_steps > 1:
            loss /= self.args.gradient_accumulation_steps
        self.accelerator.backward(loss)
        if self.args.max_grad_norm is not None:
            clip_grad_norm_(model.parameters(), self.args.max_grad_norm)
        logs = {"loss": loss.item()}
        self.log(logs)
        return loss.detach()

    def log(self, logs):
        super().log(logs)
        for key, value in logs.items():
            mlflow.log_metric(key, value, step=self.state.global_step)

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        output = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        for key, value in output.items():
            mlflow.log_metric(f"{metric_key_prefix}_{key}", value, step=self.state.global_step)

trainer = MLflowTrainer(
    class_weights_tensor=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    data_collator=custom_collate_fn,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=weight_decay_rate)
num_training_steps = training_args.max_steps
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps), 
    num_training_steps=num_training_steps
)
training_args.lr_scheduler_type = 'linear'
training_args.lr_scheduler_num_warmup_steps = int(0.1 * num_training_steps)  
mlflow.start_run()
trainer.train(resume_from_checkpoint = True)
mlflow.end_run()


# Step	Training Loss	Validation Loss	Precision	Recall	F1	Accuracy
# 50	2.172701	1.608711	0.014266	0.263959	0.027069	0.087081
# 100	1.220003	1.293408	0.021686	0.451777	0.041386	0.154461
# 150	0.910068	1.133811	0.031823	0.563452	0.060244	0.340043
# 200	1.418300	1.130773	0.034183	0.598985	0.064675	0.366310
# 250	0.557900	1.044300	0.038925	0.573604	0.072903	0.469665
# 300	0.699392	1.105067	0.056467	0.593909	0.103129	0.639971
# 350	0.343031	1.190133	0.057806	0.593909	0.105358	0.632976
# 400	0.493500	1.224606	0.072629	0.614213	0.129898	0.736759
# 450	0.281491	1.672526	0.118902	0.593909	0.198137	0.842684
# 500	0.203167	1.508098	0.094281	0.619289	0.163649	0.771592
# 550	0.153609	1.705901	0.175940	0.593909	0.271462	0.892791
# 600	0.226000	1.819784	0.128062	0.583756	0.210046	0.865525
# 650	0.045874	1.937642	0.176287	0.573604	0.269690	0.895503
# 700	0.026339	1.973493	0.226580	0.527919	0.317073	0.930050
# 750	0.040032	2.068556	0.269767	0.588832	0.370016	0.936474
# 800	0.129300	2.128340	0.223062	0.598985	0.325069	0.919914
# 850	0.053644	2.254380	0.247951	0.614213	0.353285	0.930193
# 900	0.055653	2.292882	0.252700	0.593909	0.354545	0.933476
# 950	0.046442	2.588771	0.280193	0.588832	0.379705	0.941899
# 1000	0.085500	2.407752	0.290960	0.522843	0.373866	0.945182
# 1050	0.265607	2.692389	0.400000	0.568528	0.469602	0.959315
# 1100	0.007815	2.818417	0.382022	0.517766	0.439655	0.958458
# 1150	0.009128	3.011377	0.399240	0.532995	0.456522	0.959743
# 1200	0.053700	2.781135	0.391753	0.578680	0.467213	0.958458
# 1250	0.010240	2.894034	0.374101	0.527919	0.437895	0.959029
# 1300	0.106429	3.191725	0.481481	0.527919	0.503632	0.965882
# 1350	0.062707	3.025777	0.440476	0.563452	0.494432	0.962027
# 1400	0.041300	3.192157	0.464286	0.527919	0.494062	0.965168
# 1450	0.005224	3.240162	0.448718	0.532995	0.487239	0.963740
# 1500	0.004589	3.366700	0.459459	0.517766	0.486874	0.965168

There were missing keys in the checkpoint model loaded: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias'].
There were unexpected keys in the checkpoint model loaded: ['classifier.bias', 'classifier.weight'].


ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [ ]:
# from transformers import TrainingArguments, Trainer, AdamW, get_scheduler
# import torch
# import mlflow
# from transformers import EarlyStoppingCallback
# from torch.nn import functional as F

# mlflow.end_run()

# weight_decay_rate = 0.01

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     max_grad_norm=1.0, 
#     evaluation_strategy="steps",
#     eval_steps=50,
#     save_strategy="steps",
#     save_steps=200,
#     save_total_limit=2,
#     num_train_epochs=50,
#     learning_rate=5e-4,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     fp16=True,
#     weight_decay=weight_decay_rate,
#     max_steps=1800,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     greater_is_better=True,
# )
# class MLflowTrainer(Trainer):
#     def __init__(self, class_weights_tensor, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.class_weights_tensor = class_weights_tensor

#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         # Compute Focal Loss
#         loss = self.focal_loss(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

#     def focal_loss(self, logits, labels, alpha=0.25, gamma=2, reduction='mean'):
#         cross_entropy = F.cross_entropy(logits, labels, reduction='none')
#         pt = torch.exp(-cross_entropy)
#         focal_loss = (alpha * (1 - pt) ** gamma * cross_entropy).mean()
#         return focal_loss

#     def training_step(self, model, inputs):
#         model.train()
#         inputs = self._prepare_inputs(inputs)
#         with self.autocast_smart_context_manager():
#             loss = self.compute_loss(model, inputs)
#         if self.args.gradient_accumulation_steps > 1:
#             loss /= self.args.gradient_accumulation_steps
#         self.accelerator.backward(loss)
#         if self.args.max_grad_norm is not None:
#             torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)
#         logs = {"loss": loss.item()}
#         self.log(logs)
#         return loss.detach()

#     def log(self, logs):
#         super().log(logs)
#         for key, value in logs.items():
#             mlflow.log_metric(key, value, step=self.state.global_step)

#     def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
#         output = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
#         for key, value in output.items():
#             mlflow.log_metric(f"{metric_key_prefix}_{key}", value, step=self.state.global_step)

# trainer = MLflowTrainer(
#     class_weights_tensor=class_weights_tensor,
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=processor,
#     data_collator=custom_collate_fn,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
# )

# optimizer = AdamW(model.parameters(), lr=5e-4 , weight_decay=weight_decay_rate)
# num_training_steps = training_args.max_steps
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=int(0.1 * num_training_steps), 
#     num_training_steps=num_training_steps
# )
# training_args.lr_scheduler_type = 'linear'
# training_args.lr_scheduler_num_warmup_steps = int(0.1 * num_training_steps)  
# mlflow.start_run()
# trainer.train()
# mlflow.end_run()
